## Process earnings reports data from saved pickle

E. Quinn 12/20/2020

## Import standard python datascience packages

In [1]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

### Import custom SC classes

In [3]:
sys.path.append("/home/gquinn/EG/school_committee/egsc/py_egsc")

In [4]:
from payperiod import Payperiod
from ucoa_labels import UCOA_labels
from eg_acct_codes import EG_acct_codes
from pay_check import Pay_check, Check_lineitem
from people import Person, Teacher
from scenario import Scenario
from salaries import *
from roles import Role, Teacher, Para, Sped_Para

In [5]:
salaries = Teacher_salary()
facilities_rates = Facilities_rates()
help(facilities_rates)

Help on Facilities_rates in module salaries object:

class Facilities_rates(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Salaries and rates from facilities collective bargaining agreement
 |  
 |  get_cba_matrix(self)
 |      Returns the salary and rate matrix
 |  
 |  get_cba_matrix_by_year(self, school_year)
 |      Returns the cba salary and rate matrix for a given school year
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### Show the directory we are running in

In [6]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


### Load RIDE UCOA labels 

In [7]:
ucoa_labels = UCOA_labels()
    
help(ucoa_labels)

Help on UCOA_labels in module ucoa_labels object:

class UCOA_labels(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      UCOA_labels class provides labels for UCOA fields
 |  
 |  get_label(self, col, code)
 |      Usage: get_label(col,code) returns the code for field col
 |  
 |  get_labels_dictionary(self)
 |      Returns the UCOA labels dictionary
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### EG accounting codes class

provides descriptions for EG accounting codes and mapping to UCOA codes

In [8]:
eg_acct_codes = EG_acct_codes()
    
help(eg_acct_codes)

Help on EG_acct_codes in module eg_acct_codes object:

class EG_acct_codes(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  check_eg_acct_desc6(self, acct)
 |      check whether first 6 digits map an acct6 key
 |  
 |  get_UCOA_from_acct(self, acct, ucl)
 |  
 |  get_UCOA_from_acct6(self, acct6, ucl)
 |  
 |  get_eg_acct_UCOA(self, acct)
 |      Provides UCOA codes for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_codes(self)
 |      Returns dictionary of account codes.
 |  
 |  get_eg_acct_desc(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_desc6(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_loc_from_acct(self, acct, ucl)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary 

### Load serialized people object produced from earnings report

In [9]:
#Load data (deserialize)
with open('../../finance_subcommittee/people_12_27_2020.pkl', 'rb') as handle:
    people = pickle.load(handle)
    
len(people)

898

In [10]:
file1 = open('../facilities_earnings_detail_12_27_2020.csv','w')

hstr = '"Name","Role","School Year","Fiscal Year","Payperiod Number",' + \
    '"Check Date","Check Number","Payment Type","Fund","Acct","Acct Desc","Obj",' +\
    '"Obj Desc","Rate","Earnings","Job","O/T","Earnings","Rate","Hours"\n'
file1.write(hstr)

for name in people.keys():
    hstr = '"' + name + '",'
    roles = people[name].get_roles()
    for role in roles.keys():
        if (str(type(roles[role])) == "<class 'roles.Facilities'>"):
            hstr2 =  hstr + '"' + role + '","'
            pp = roles[role].get_payperiods()
            for syr in pp.keys():
                hstr3 = hstr2 + syr + '",'
                for pseq in pp[syr].keys():
                    fyear = pp[syr][pseq].get_fiscal_year()
                    hstr4 = hstr3 + str(fyear) + ',' + str(pseq) + ','
                    paychecks = pp[syr][pseq].get_checks()
                    for ckey in paychecks.keys():
                        check_date = paychecks[ckey].get_date()
                        check_number= paychecks[ckey].get_number()
                        hstr5 = hstr4 + str(check_date) + ',' + str(check_number)
                        itms = paychecks[ckey].get_items()
                        for itmkey in itms.keys():
                            fund = itms[itmkey].get_fund()
                            acct = itms[itmkey].get_acct()
                            obj  = itms[itmkey].get_obj()
                            rate = itms[itmkey].get_rate()
                            accdesc = itms[itmkey].get_acct_desc()
                            objdesc = itms[itmkey].get_obj_desc()
                            earnings = itms[itmkey].get_earnings()
                            payment_type = itms[itmkey].get_payment_type()
                            stepinfo = itms[itmkey].get_stepinfo()
                            hstr6 = hstr5 + ',' + str(payment_type) + ',' + fund + ',' + \
                                acct + ',' + accdesc + ',' + obj + ',' + objdesc + ',' + \
                                str(rate) + ',' + str(earnings)
                            hstr7 = hstr6
                            if (stepinfo is not None): 
                                if(len(stepinfo) > 1):
                                    job = stepinfo['job']
                                    overtime = stepinfo['overtime']
                                    si_earnings = stepinfo['earnings']
                                    si_rate = stepinfo['rate']
                                    hours = stepinfo['hours']
                                    hstr7 = hstr6 + ',' + job + ',' + str(overtime) + ',' + \
                                        str(si_earnings) + ',' + str(si_rate) + ',' + \
                                    str(hours)
                            hstr7 = hstr7 + '\n'
                            file1.write(hstr7)
file1.close()